# load data from pkl file 

# print the number of "spurious examples" extracted

# print the sentence associated

# load the id of extracted examples and show if they are in an other pkl file

In [11]:
!pip install pandas

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [12]:
import pickle

# Charger les exemples extraits
with open('mnli_bert_base/important_examples.pkl', 'rb') as file:
    extracted_examples = pickle.load(file)

# Vérifier le type de la structure de données
print(f"Type de la structure chargée : {type(extracted_examples)}\n")

# Afficher les premières entrées selon le type de données
if isinstance(extracted_examples, list):
    # Si c'est une liste
    for i, item in enumerate(extracted_examples[:5]):
        print(f"Élément {i+1} : {item}\n")

# good one
elif isinstance(extracted_examples, dict):
    # Si c'est un dictionnaire
    for i, (key, value) in enumerate(extracted_examples.items()):
        if i >= 5:
            break
        print(f"Clé : {key}\nValeur : {value}\n")

elif isinstance(extracted_examples, (set, tuple)):
    # Si c'est un ensemble ou un tuple
    for i, item in enumerate(list(extracted_examples)[:5]):
        print(f"Élément {i+1} : {item}\n")

else:
    # Si c'est un autre type de données
    print(extracted_examples)
print(len(value))


Type de la structure chargée : <class 'dict'>

Clé : selected_examples
Valeur : [     3      4     61 ... 392663 392664 392665]

26364


In [18]:
import pickle
import pandas as pd

# Charger les indices d'exemples sélectionnés
with open('mnli_bert_base/important_examples.pkl', 'rb') as file:
    extracted_examples = pickle.load(file)

selected_indices = extracted_examples['selected_examples']

# Charger le fichier train.tsv
# train_data = pd.read_csv('mnli/MNLI/MNLI/train.tsv', sep='\t')
train_data =  pd.read_csv('mnli/MNLI/MNLI/train.tsv',
                        delimiter='\t',
                        on_bad_lines='skip', 
                        skiprows=0,
                        quoting=3,
                        keep_default_na=False,
                        encoding="utf-8",)
# Vérifier la structure des colonnes
print(f"Colonnes disponibles : {train_data.columns.tolist()}")

extracted_data = []

for idx in selected_indices:  # Limité aux 5 premiers exemples
    row = train_data.iloc[idx]
    extracted_data.append({
        'Index': idx,
        'Sentence1': row['sentence1'],
        'Sentence2': row['sentence2'],
        'Gold_Label': row['gold_label']
    })

# Enregistrer les données extraites dans un fichier CSV
output_df = pd.DataFrame(extracted_data)
output_df.to_csv('extracted_examples.csv', index=False)

print("Les phrases extraites ont été enregistrées dans 'extracted_sentences.csv'.")


Colonnes disponibles : ['index', 'promptID', 'pairID', 'genre', 'sentence1_binary_parse', 'sentence2_binary_parse', 'sentence1_parse', 'sentence2_parse', 'sentence1', 'sentence2', 'label1', 'gold_label']
Les phrases extraites ont été enregistrées dans 'extracted_sentences.csv'.


In [ ]:
import pickle
import pandas as pd

# Charger le dataset MNLI
mnli_data = pd.read_csv('train.tsv', sep='\t')

# Fonction pour analyser la distribution des labels
def analyze_label_distribution(pkl_file, method_name):
    with open(pkl_file, 'rb') as file:
        extracted_indices = pickle.load(file)
    
    # Filtrer les exemples extraits
    filtered_data = mnli_data[mnli_data['index'].isin(extracted_indices)]
    
    # Compter la distribution des labels
    label_counts = filtered_data['gold_label'].value_counts()
    print(f"Distribution des labels pour {method_name}:")
    print(label_counts)
    print('-' * 40)
    return label_counts

# Appliquer l'analyse pour chaque méthode
att_labels = analyze_label_distribution('attention_score.pkl', 'Attention Score')
lid_labels = analyze_label_distribution('lid_score.pkl', 'LID Score')
forget_labels = analyze_label_distribution('forget_events.pkl', 'Forget Events')


In [ ]:
import matplotlib.pyplot as plt

# Combiner les résultats dans un DataFrame
label_distribution = pd.DataFrame({
    'Attention Score': att_labels,
    'LID Score': lid_labels,
    'Forget Events': forget_labels
}).fillna(0)

# Plot
label_distribution.plot(kind='bar', figsize=(10, 6))
plt.title('Distribution des Labels par Méthode d’Extraction')
plt.xlabel('Classe (gold_label)')
plt.ylabel('Nombre d\'Exemples')
plt.xticks(rotation=0)
plt.show()
